## testing rero2 boxes

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime
import dask.bag as db
import glob

In [2]:
from impresso_commons.path.path_fs import IssueDir, detect_canonical_issues, canonical_path, KNOWN_JOURNALS
from text_importer.helpers import get_page_schema
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT

In [3]:
from impresso_commons.images.olive_boxes import get_iiif_url
from IPython.display import Image, display, HTML, Markdown

required input: canonical data before compression and upload; here tested with Edoardo's files 

In [25]:
%%time
page_files = list(filter(lambda f: 'issue' not in f, list(glob.iglob('/scratch/students/rero2/BLB/**/*.json', recursive=True))))
len(page_files)

CPU times: user 7.14 s, sys: 4.65 s, total: 11.8 s
Wall time: 6min 43s


749944

In [21]:
page_files

['/scratch/students/rero2/BLB/1845/12/28/a/BLB-1845-12-28-a-p0002.json',
 '/scratch/students/rero2/BLB/1845/12/28/a/BLB-1845-12-28-a-p0001.json',
 '/scratch/students/rero2/BLB/1845/12/28/a/BLB-1845-12-28-a-p0003.json',
 '/scratch/students/rero2/BLB/1845/12/28/a/BLB-1845-12-28-a-p0004.json',
 '/scratch/students/rero2/BLB/1845/12/28/a/BLB-1845-12-28-a-p0005.json',
 '/scratch/students/rero2/BLB/1846/06/06/a/BLB-1846-06-06-a-p0003.json',
 '/scratch/students/rero2/BLB/1846/06/06/a/BLB-1846-06-06-a-p0004.json',
 '/scratch/students/rero2/BLB/1846/06/06/a/BLB-1846-06-06-a-p0001.json',
 '/scratch/students/rero2/BLB/1846/06/06/a/BLB-1846-06-06-a-p0002.json',
 '/scratch/students/rero2/BLB/1846/06/13/a/BLB-1846-06-13-a-p0002.json',
 '/scratch/students/rero2/BLB/1846/06/13/a/BLB-1846-06-13-a-p0004.json',
 '/scratch/students/rero2/BLB/1846/06/13/a/BLB-1846-06-13-a-p0003.json',
 '/scratch/students/rero2/BLB/1846/06/13/a/BLB-1846-06-13-a-p0001.json',
 '/scratch/students/rero2/BLB/1846/06/20/a/BLB-1846

In [26]:
import os
import json
from random import randint, shuffle

def select_random_index(objects):
    shuffle(objects)
    total_objects = len(objects)
    index = randint(0, len(objects) - 1)
    return index

def select_random_token(page_file):
    with open(page_file) as json_file:  
        page_json = json.load(json_file)
        total_regions = len(page_json['r'])
        region = page_json['r'][randint(0, total_regions - 1)]
        total_paragraphs = len(region['p'])
        paragraph = region['p'][randint(0, total_paragraphs - 1)]
        total_lines = len(paragraph['l'])
        line = paragraph['l'][randint(0, total_lines - 1)]
        total_tokens = len(line['t'])
        token = line['t'][randint(0, total_tokens - 1)]
        return token
    
def select_random_region(page_file):
    with open(page_file) as json_file:  
        page_json = json.load(json_file)
        total_regions = len(page_json['r'])
        region = page_json['r'][randint(0, total_regions - 1)]
        return region

def get_link(page_name, coordinates):
    link = get_iiif_url(page_name, " ".join([str(i) for i in coordinates]))
    return link

def get_image(page_name, coordinates, user="epfl-team", pwd="_pYmypGG#+N;6{3U"):
    link = get_iiif_url(page_name, " ".join([str(i) for i in coordinates]))
    return requests.get(link, auth=HTTPBasicAuth(user, pwd)).content
    

In [27]:
import re
import pandas as pd
for n in range(0, 200):
    page_idx = select_random_index(page_files)
    page = page_files[page_idx]
    #print(page)
    for i in range(0, 2):
        try:
            token = select_random_token(page)
            basename = page[re.search(r"[A-Z]{3,3}-", page).start():].replace('.json', '')
            link_token = get_link(basename, token['c'])
            print(f"{token['tx']} \t\t\t\t\t// ({basename}) ({token['c']})")
            display(Image(url=link_token))
            print("-----------------\n")
                  
            #reg = select_random_region(page)
            #link_reg = get_link(basename, reg['c'])
            #print(basename)
            #display(Image(url=link_reg))
            #display(Image(url=link_reg.replace("iiif_impresso","iiif_impresso_new")))
            #print("-----------------\n")
            
        except Exception as e:
            print(e)
            pass

ranon 					// (LLE-1939-03-04-a-p0005) ([2807, 1025, 90, 48])


-----------------

explosion 					// (LLE-1939-03-04-a-p0005) ([1693, 4549, 161, 48])


-----------------

ItUlfIMDIUlt 					// (LLE-1987-07-14-a-p0004) ([2899, 2379, 393, 98])


-----------------

a 					// (LLE-1987-07-14-a-p0004) ([1997, 4611, 52, 98])


-----------------

empty range for randrange() (0,0, 0)
empty range for randrange() (0,0, 0)
empty range for randrange() (0,0, 0)
^^ 					// (LLE-1994-11-05-a-p0040) ([2242, 2600, 290, 271])


...

-----------------

apparte- 					// (LLE-1997-07-26-a-p0006) ([1341, 1136, 123, 54])


-----------------

rédact 					// (LLE-2009-09-23-a-p0003) ([3329, 4432, 93, 38])


-----------------

marque 					// (LLE-2009-09-23-a-p0003) ([1278, 2653, 137, 60])


-----------------



KeyboardInterrupt: 